## This notebook is to automate the creation of the data populationsheet

### the final result is written in the following box folder. 

https://ibm.ent.box.com/folder/117380324021

In [ ]:
# default is false, 
# when false, it will calculate the result based on all record (A, I, N)
# when true, it will only calcualte based on active emplyee E01.CACTIVE = 'A'
### change on 2020/11/18
active_flag = False

In [ ]:
na_flag = '--' if not active_flag else ''
na_flag
'_active' if active_flag else ''

In [ ]:
import sys
import time
sys.path.append('/odm_modules')
from common_func import cloudant_conn as cc
from common_func import odm_conn
from common_func import odm_dict
import pandas as pd
import logging                                                                  
import os  
sys.path.append('/app')
from BOX import box_oauth as box 
import openpyxl
import datetime
from openpyxl.styles import Color, PatternFill, Font, Border, Alignment
from openpyxl.styles import colors
from openpyxl.cell import Cell
logging.basicConfig(stream=sys.stdout, level=20) 
from openpyxl.comments import Comment

In [ ]:
cc.cloudant_client.connect()
odmmeta_api_root = 'https://odmmeta.dal1a.ciocloud.nonprod.intranet.ibm.com/igc/get_term/{}/format=text'
df_busi= pd.DataFrame(list(cc.cloudant_client['busi_def']))
busi_field_list = list(df_busi._id)
busi_dict = df_busi.set_index('_id')['long_description'].to_dict()

In [ ]:
with odm_conn.odm_adhoc('prod') as odmprd_adhoc:
    result = odmprd_adhoc('''
    select CISOCTRY,TISOCTRY from odmprd.odmt_countryiso 
    where CLANGUAG = '' AND FDISCONT <> 'Y' ''')
    result_fdr = odmprd_adhoc('''
    select cfdrsrc, tfdrsrc from odmprd.ODMT_FEEDSRCE
    where clanguag = ''
    ''')

df_ctry = pd.DataFrame(result).applymap(lambda x: x.strip())
ctry_dict = df_ctry.set_index('CISOCTRY').TISOCTRY.to_dict()

df_fdr = pd.DataFrame(result_fdr).applymap(lambda x: x.strip())
fdr_dict = df_fdr.set_index('CFDRSRC').TFDRSRC.to_dict()
#fdr_dict['HD1']

In [ ]:
# na_flag is to switch if we will use the condition CACTIVE = A or not 
### change on 2020/11/18
sql_temp = '''
SELECT {tid}.COUNTRY 
{field_sum_list} 
, COUNT(*) AS TOTAL
FROM 
(SELECT A.CCOUNTRY AS COUNTRY, A.* FROM ODMPRD.{tb_name} A, ODMPRD.ODMT_EMPLOYEE E01 
WHERE 
E01.CCOUNTRY = A.CCOUNTRY
AND E01.CCOUNTRQ = A.CCOUNTRQ
AND E01.RSERNUM = A.RSERNUM 
{na_flag} AND E01.CACTIVE = 'A' 
) {tid}
GROUP BY {tid}.COUNTRY ;
'''
sql_temp_fdr = '''
SELECT {tid}.FDRSRC 
{field_sum_list} 
, COUNT(*) AS TOTAL
FROM (
SELECT E01.CFDRSRC AS FDRSRC, 
A.* FROM 
ODMPRD.ODMT_EMPLOYEE E01, 
ODMPRD.{tb_name} A
WHERE 
E01.RSERNUM = A.RSERNUM
AND E01.CCOUNTRY = A.CCOUNTRY
AND E01.CCOUNTRQ = A.CCOUNTRQ
{na_flag} AND E01.CACTIVE = 'A' 
) {tid}
GROUP BY {tid}.FDRSRC ;
'''

sql_temp_d = '''
SELECT {tid}.COUNTRY 
{field_sum_list} 
,COUNT(*) AS TOTAL
FROM 
(SELECT CASE                                                         
WHEN E1.CCOUNTRY IS NULL THEN SUBSTR('  ', 1, 2)             
ELSE E1.CCOUNTRY END AS COUNTRY, A.*                         
FROM ODMPRD.{tb_name} A                                            
LEFT OUTER JOIN                                              
ODMPRD.ODMT_EMPLOYEE E1                                      
ON E1.RCNUM = A.RCNUM) {tid}                                   
GROUP BY {tid}.COUNTRY ;           
'''
field_sum_temp = '''\n, SUM(CASE WHEN {tid}.{field} = '{dft_value}' THEN 0 ELSE 1 END) AS {field} '''

In [ ]:
odm_dict.refresh()

In [ ]:
df_tid = odm_dict.odm_tid_df_from_csv()
df_col = odm_dict.odm_col_df_from_csv()
df_col = df_col.loc[df_col.NAME !=''] # remove those line which could not be found in systable
df_tid = df_tid.loc[df_tid.NAME !='']
tid_dict = df_tid.set_index('CTID')['CTABNAME'].to_dict()
tid_desc_dict = df_tid.set_index('CTID')['TTID'].to_dict()
df_col['tid_col'] = df_col.CTID + '.' + df_col.CCOLNAME
data_class_dict = df_col.set_index('tid_col').CCLASS.to_dict()

In [ ]:
def get_population(tid, sql_template, idx_col = 'COUNTRY'):
    logging.info('*'*10 + ' TID is {}'.format(tid))
#    drop_list = ['CCOUNTRY', 'CCOUNTRQ', 'RSERNUM', 'CODM',  'CFDRSRC' ]
#    drop_list = [ 'CODM', 'CCOUNTRQ', 'CFDRSRC' ]
    drop_list = [ ]
    tb_name = tid_dict[tid]
    col_list = list(df_col.loc[df_col.CTID == tid].sort_values(by = 'COLNO')['CCOLNAME'])
    col_list_filtered = list(filter(lambda field: field not in drop_list and 'SEQ' not in field, col_list))
    field_sum_list =''.join( [field_sum_temp.format(
        tid = tid, field = field, 
        dft_value = '0001-01-01' 
        if df_col.loc[(df_col.CTID == tid) & (df_col.CCOLNAME == field)].COLTYPE.values[0] == 'DATE'
        else 0 if df_col.loc[(df_col.CTID == tid) & (df_col.CCOLNAME == field)].COLTYPE.values[0] in ['DECIMAL', 
                                                                                                      'SMALLINT', 
                                                                                                      'INTEGER', 
                                                                                                      'BIGINT']
        else '' 
    ) for field in col_list_filtered])
    ###
    ### change on 2020/11/18, added na_flag
    sql = sql_template.format(tid = tid, field_sum_list = field_sum_list, tb_name = tb_name, na_flag = '--' if not active_flag else '')
    logging.debug(sql)

    with odm_conn.odm_adhoc('prod') as odmprd_adhoc:
        result = odmprd_adhoc(sql)

    df = pd.DataFrame(result, columns = [idx_col, *col_list_filtered, 'TOTAL']).set_index(idx_col)
    #print(df)
    #print(df.sum)
    df.loc['Total'] = df.sum()   
    
    # caculate the percentage
    df['TEMP'] = df.RCNUM if 'RCNUM' in df.columns else df.RSERNUM
    for col in [x for x in df.columns if x != 'TEMP']:
        df[col] = df[col]/df.TEMP
    df.drop(columns = ['TEMP'], inplace = True)
    #pd.options.display.float_format = '{:.0f}%'.format
    # enrich with country description and field desc
##x    col_dict = df_col.loc[(df_col.CTID == tid), ['CCOLNAME', 'TCOLNAME']].set_index('CCOLNAME').TCOLNAME.to_dict()
##x    df.columns = pd.MultiIndex.from_tuples([(col, col_dict.get(col,''), 
##x                                            data_class_dict.get(tid + '.' + col, ''))  
##x                                            for col in df.columns], names = (None, None, None))
    if not df.empty:
        # reorder the fdr list
        if idx_col == 'FDRSRC':
            fdr_ordered_list = '''
            HC9	HHA	HHE	HHI	HHJ	HHP	HNW	HPH	HSW	HZA	HEU	HJ1	
            HBM	HB8	HB9	HCE	HC8	HD1	HD8	HE2	HF4	HF8	HF9	HIN	
            HI7	HI8	HKR	HOG	HRH	HSK	HS1
            '''.split()
            #df_temp = pd.DataFrame(columns = ['Total', *fdr_ordered_list]).T # create an empty df
            if tid == 'E01': # only reorder E01 table and other table will be aligned to E01 table. 
                ### 2020/11/18 change
                #df = df.loc[['Total', *fdr_ordered_list]]
                df = df.reindex(index = ['Total', *fdr_ordered_list] )
            #df = pd.concat([df_temp, df], axis = 1)
        else: 

            temp_idx = list(df.index)
            temp_idx.remove('Total')
            ### 2020/11/18 change
            #df = df.loc[['Total', *temp_idx]]
            df = df.reindex(index = ['Total', *temp_idx])
        # determine which dictionary to be used based on index column name
        df.index = [ 'blank' if idx.strip() == '' else idx   for idx in df.index] # rename the blank value to be string blank
##x        mapping_dict = fdr_dict if idx_col == 'FDRSRC' else ctry_dict 
##x        df.index = pd.MultiIndex.from_tuples([(ctry_fdr, mapping_dict.get(ctry_fdr,''))  for ctry_fdr in df.index])
    return df

In [ ]:
# this function should only used when finally write to excel file
def add_desc(tid, df, idx_col = 'COUNTRY'): # add desc into index and columns using disctionary
    col_dict = df_col.loc[(df_col.CTID == tid), ['CCOLNAME', 'TCOLNAME']].set_index('CCOLNAME').TCOLNAME.to_dict()
#    print(col_dict)
#    print(df.columns)
    
    df.index = pd.MultiIndex.from_tuples([(col, col_dict.get(col,''), 
                                            data_class_dict.get(tid + '.' + col, ''))  
                                            for col in df.index], names = (None, None, None))
    if not df.empty:
        mapping_dict = fdr_dict if idx_col == 'FDRSRC' else ctry_dict 
        df.columns = pd.MultiIndex.from_tuples([(ctry_fdr, mapping_dict.get(ctry_fdr,''))  for ctry_fdr in df.columns])
    return df

In [ ]:
tid_list = list(filter(lambda x: x!='' 
                       and x[0] in ['D']
                       and x not in ['D19', 'D20']
#                       and x in ['D01', 'D02']  
                       , list(df_tid.CTID.drop_duplicates())))
logging.info(str(tid_list))
dfs_d ={tid: get_population(tid, sql_temp_d).T.fillna(0) for tid in tid_list}

In [ ]:
tid_list = list(filter(lambda x: x!='' 
                       and x[0] in ['E', 'F']
#                       and x == 'E03'
                       ,list(df_tid.CTID.drop_duplicates())))
logging.info(str(tid_list))
dfs_ef ={tid: get_population(tid, sql_temp).T.fillna(0) for tid in tid_list}
#tid_list = list(filter(lambda x: x!='' 
#                       and x[0] in ['E', 'F']
#                       and x in ['E01', 'E02', 'E03', 'F01']
#                       ,list(df_tid.CTID.drop_duplicates())))
dfs_fdr = {tid: get_population(tid, sql_temp_fdr, idx_col = 'FDRSRC').T.fillna(0) for tid in tid_list}

### The following code is to generate all the sample value for each field. 

1. We only check non PI data, which started with R, T, C and it is char type
2. when the count of that field is less then 40, we try to get the first 10 most frequent sample values from the database

In [ ]:
dfs_d['D03']

In [ ]:
# get the count of distinct values
sql_temp_cnt = '''
select 
{col_cnt_list}
from odmprd.{tbname} {tid}
'''

def get_cnt(tid): 
    tbname = tid_dict.get(tid, '')
    ### following the filter is to get all the in-scope columns from the list
    col_list_all = df_col.loc[(df_col.CTID == tid) & 
                              (((df_col.CCLASS == 'O') & df_col.CCOLNAME.apply(lambda x: len(x) > 1 and x[0] in ['R', 'T', 'C'] ) ) 
                              | df_col.CCOLNAME.apply(lambda x: len(x) > 1 and x[0] in [ 'C', 'F'] ))
                              & (df_col.CCOLNAME != 'CSPSSOCS')  # Remove one SSN field
                          & (df_col.COLTYPE == 'CHAR')
                         ].CCOLNAME
    
    col_list_list = [col_list_all[x:x+3] for x in range(0, len(col_list_all), 3)]
    print(col_list_list)
    
    with odm_conn.odm_adhoc('prod') as odmprd_adhoc: 
        def chunk_count(col_list): 
            col_cnt_list = ["COUNT(DISTINCT {}) AS {}".format(col, col) for col in col_list]
            col_cnt_list = '\n,'.join(col_cnt_list)
            sql = sql_temp_cnt.format(col_cnt_list = col_cnt_list, tbname = tbname, tid = tid)
            print(sql)
            result = odmprd_adhoc(sql)
            df_temp = pd.DataFrame(result)
            df_temp.index = ['distinct_cnt']
            return df_temp

        df = pd.concat([chunk_count(col_list)  for col_list in col_list_list] 
                       if len(col_list_list) !=0 else [pd.DataFrame()], axis = 1)
    
    return tid, df

# only D table and E table will be checked. 
#dfs = dict([get_cnt(tid) for tid in tid_dict.keys() if tid != '' and tid[0:3] in ['E02', 'D01']])
dfs = dict([get_cnt(tid) for tid in tid_dict.keys() if tid != '' and tid[0] in ['E', 'D']])

In [ ]:
# get the first 10 most frequent values 

def get_distinct_value(tid, threshhold):
    if dfs[tid].empty:
        return ''
    col_list = [x[0] for x in dfs[tid].iloc[0].items() if x[1]<=threshhold or threshhold == 0]
    sql_temp_distinct = '''
    select {col}, count(*) AS CNT from odmprd.{tbname} {tid} 
    where {col} <> ''
    group by {col}
    order by count(*) DESC
    fetch first 10 rows only 
    '''
    sqls = [sql_temp_distinct.format(tid = tid, tbname = tid_dict[tid], col=col) for col in col_list]
    for sql in sqls: 
        print(sql)
    with odm_conn.odm_adhoc('prod') as odmprd_adhoc:
        results = [odmprd_adhoc(sql) for sql in sqls]
        df_tmps = [pd.DataFrame(result) for result in results]
        df_tmps = [df.drop(columns = ['CNT']).applymap(lambda x: x.strip()) for df in df_tmps if not df.empty]

    ser = pd.Series( {df.columns[0]: '; '.join(df.values.flat) for df in df_tmps})
    return ser

# 40 is the threshold
# if threshhold == 0, then it means no threshhold
#sers = {tid: get_distinct_value(tid, 40) for tid in dfs.keys()}
sers = {tid: get_distinct_value(tid, 0) for tid in dfs.keys()}

In [ ]:
dfs_distinct = dict()
for tid, df in dfs.items():
    df = df.T
    df['Top 10 non-blank values'] = sers[tid]
    dfs_distinct[tid] = df.fillna('')
    if not dfs_distinct[tid].empty:
        #dfs_distinct[tid].distinct_cnt = dfs_distinct[tid].distinct_cnt.astype(int)
        dfs_distinct[tid].distinct_cnt = dfs_distinct[tid].distinct_cnt.apply(lambda x: '<=10' if x<=10 else '<=40' if x<=40 else '>40' )
#dfs_distinct = pd.concat(dfs_distinct.values())
dfs_distinct['E01']

In [ ]:
xlsx2_file = '10 top distinct values.xlsx'
with pd.ExcelWriter(xlsx2_file) as writer: 
    for tid, df in dfs_distinct.items():
        df.to_excel(writer, sheet_name=  tid )

### Concat the dfs_ef dfs_d df_fdr with dfs_distinct to get the dfs_final

In [ ]:
# concat the distinct count and sample values
dfs_final = dict()
dfs_fdr_final = dict()

for tid, df in [ *(dfs_ef.items()), *(dfs_d.items())]:
    dfs_final[tid] = pd.concat([df, dfs_distinct.get(tid,pd.DataFrame(columns = ['distinct_cnt', 'Top 10 non-blank values']) )],
                               axis = 1)
for tid, df in dfs_fdr.items(): 
    dfs_fdr_final[tid] = pd.concat([df , dfs_distinct.get(tid,pd.DataFrame(columns = ['distinct_cnt', 'Top 10 non-blank values'])) ], 
                           axis = 1)

#dfs_final ={tid: add_desc(tid, df) if tid != 'FDR' 
#            else add_desc('E01', df_fdr_new, idx_col = 'FDRSRC')
#            for tid, df in [('FDR', df_fdr_new)  , *dfs_final.items()]}

dfs_final = {tid: add_desc(tid, df) for tid, df in dfs_final.items()}
dfs_fdr_final = {'FDR_{}'.format(tid): add_desc(tid, df, idx_col = 'FDRSRC') for tid, df in dfs_fdr_final.items()}
dfs_final = {**dfs_final, **dfs_fdr_final} 
#dfs_final['FDR'] = add_desc('E01', df_fdr_new, idx_col = 'FDRSRC')
#df_exx = pd.concat([dfs_final[tid] ).fillna(0)

In [ ]:
# merge those E, F table into one single sheet
dfs_efx = []
for tid in filter(lambda x: x[0] in ['E' ,'F'] and 'FDR' not in x , dfs_final.keys()):
    df = dfs_final[tid].copy()
    df = df.loc[filter(lambda idx: idx != ('TOTAL', '', ''),df.index)] # remove the total line since it is meaningless
    df.index =pd.MultiIndex.from_tuples([(*idx, tid)   for idx in df.index])
    dfs_efx.append(df)
    
df_exx = pd.concat(dfs_efx)
df_exx = df_exx[dfs_final['E01'].columns]
df_exx

dfs_fdr_merged = []
for key in filter(lambda x: 'FDR' in x , dfs_final.keys()):
    df = dfs_final[key].copy()
    df = df.loc[filter(lambda idx: idx != ('TOTAL', '', ''),df.index)] # remove the total line since it is meaningless
    df.index =pd.MultiIndex.from_tuples([(*idx, key[-3:])   for idx in df.index])
    dfs_fdr_merged.append(df)
    
df_fdr = pd.concat(dfs_fdr_merged)
df_fdr = df_fdr[dfs_final['FDR_E01'].columns]
df_fdr

In [ ]:
# df_fdr_copy is the summary table
df_fdr_copy = df_fdr.copy()
df_fdr_copy.index.names = ('A', "B", "C", 'TID')
df_fdr_copy.reset_index(inplace = True)
df_fdr_copy = df_fdr_copy.drop(columns = [('A'), ('B', ), ('C', ), ('Total', )])
df_fdr_copy = df_fdr_copy.groupby('TID').agg(lambda s: '' if s.isnull().all() else s.mean())
df_fdr_copy.index = pd.MultiIndex.from_tuples( (x, tid_dict.get(x, ''), tid_desc_dict.get(x, ''))   for x in df_fdr_copy.index)

df_exx_copy = df_exx.copy()
df_exx_copy.index.names = ('A', "B", "C", 'TID')
df_exx_copy.reset_index(inplace = True)
df_exx_copy = df_exx_copy.drop(columns = [('A'), ('B', ), ('C', ), ('Total', )])
df_exx_copy = df_exx_copy.groupby('TID').agg(lambda s: '' if s.isnull().all() else s.mean())
df_exx_copy.index = pd.MultiIndex.from_tuples( (x, tid_dict.get(x, ''), tid_desc_dict.get(x, ''))   for x in df_exx_copy.index)


#df_fdr_copy.iloc[:, [0,1]]
#df_fdr_copy.iloc[1057, 1].isnull()

In [ ]:
###
### change on 2020/11/18
###
xlsx_file = 'data_population_masked_percentage{}.xlsx'.format('_active' if active_flag else '')
with pd.ExcelWriter(xlsx_file) as writer: 
    for tid, df in[('EFx', df_exx), ('FDR', df_fdr), ('FDRSUM', df_fdr_copy), ('ExxSUM', df_exx_copy), *dfs_final.items()]:
        df.to_excel(writer, sheet_name=  tid )

### Render the data with color coding

In [ ]:
# small change in the layout of the sheet
xfile = openpyxl.load_workbook(xlsx_file)
for tid in xfile.sheetnames:
    sheet = xfile[tid]
    sheet.insert_rows(1)
    sheet.delete_rows(4,1)
xfile.save(xlsx_file)

In [ ]:
# make the color coding on the sheet. no tweak on the existing cells. 
cur_time = str(datetime.datetime.now())
greenFill = PatternFill(start_color='CCCCFF11',
               end_color='CCCCFF11',
               fill_type='solid')
lightRedFill = PatternFill(start_color='00EEAAEE',
               end_color='00EEAAEE',
               fill_type='solid')
redFill = PatternFill(start_color='00EE6600',
               end_color='00EE6600',
               fill_type='solid')
colFill = PatternFill(start_color='CCCCCCFF',
               end_color='CCCCCCFF',
               fill_type='solid')
rowFill = PatternFill(start_color='AAEEEECC',
               end_color='AAEEEECC',
               fill_type='solid')

greyFill = PatternFill(start_color='CCCCCCCC',
               end_color='CCCCCCCC',
               fill_type='solid')
naFill = PatternFill(start_color='EECCFFFF',
               end_color='EECCFFFF',
               fill_type='solid')
color = ['00DDDDDD', '00CCCCCC', '00AAAAAA' ,'00888888', '00666666'  ]
colorFill = [PatternFill(start_color=c, end_color=c,fill_type='solid') for c in color]

xfile = openpyxl.load_workbook(xlsx_file)
for tid in xfile.sheetnames:
    sheet = xfile[tid]
    #sheet.insert_rows(1)
    #tid = 'E01' if tid == 'FDR' else tid
    sheet['A1'] = tid
    sheet['B1'] = tid_dict.get(tid if 'FDR' not in tid  else tid[-3:] , '') 
    sheet['C1'] = tid_desc_dict.get(tid if 'FDR' not in tid  else tid[-3:] , '') + ':' + cur_time
    #sheet['D1'] = cur_time
    sheet['A3'] = 'ODM Field' if tid not in  ['FDRSUM', 'ExxSUM'] else 'TID'
    sheet['B3'] = 'Description' if tid not in  ['FDRSUM', 'ExxSUM'] else 'Table Name'
    sheet['C3'] = 'Data class' if tid not in  ['FDRSUM', 'ExxSUM'] else 'Table description'
    if tid in ['EFx', 'FDR']:
        sheet['D3'] = 'TID'

    # delete the empty row
    #sheet.delete_rows(4,1)

    # fill color
    for row in sheet.iter_rows(min_row=1, max_col=sheet.max_column, max_row=1):
        for cell in row:
            cell.fill = greenFill

    # fill grey every other row
#    switch = 0
#    for row in sheet.iter_rows(min_row=4, max_col= sheet.max_column, min_col = 3 , max_row=sheet.max_row):
#        if switch == 0: 
#            for cell in row:
#                cell.fill = greyFill
#            switch = 1
#        else:
#            switch = 0

    # the contents, max_col -2 is to ignore the last 2 columns which are for distinct count and sample values
    for row in sheet.iter_rows(min_row=4, 
                               max_col=sheet.max_column-2 if tid not in  ['FDRSUM', 'ExxSUM'] else sheet.max_column, 
                               max_row=sheet.max_row, 
                               min_col = 4 if tid not in ['EFx', 'FDR'] else 5):
        for cell in row:
            cell.alignment = Alignment(horizontal='right')

            cell.font = Font(color = '000000')
            if isinstance(cell.value, (int, float)):
                if float(cell.value) == 0:
                    cell.fill = PatternFill(start_color='00FFFFFF', end_color='00FFFFFF',fill_type='solid')
                    cell.number_format = '0'
                elif float(cell.value) < 0.2: 
                    cell.fill = colorFill[0]
                    cell.number_format = '0.00%'
                elif float(cell.value) < 0.4: 
                    cell.number_format = '0.00%'
                    cell.fill = colorFill[1]
                elif float(cell.value) < 0.6: 
                    cell.font = Font(color = 'FFFFFF')
                    cell.number_format = '0.00%'
                    cell.fill = colorFill[2]
                elif float(cell.value) < 0.8: 
                    cell.number_format = '0.00%'
                    cell.font = Font(color = 'FFFFFF')
                    cell.fill = colorFill[3]
                else: 
                    cell.number_format = '0.00%'
                    cell.fill = colorFill[4]
                    cell.font = Font(color = 'FFFFFF')
            else : 
                cell.fill = naFill

    # this is for the columns distinct_cnt and top 10 non blank values columns
    if tid not in  ['FDRSUM', 'ExxSUM']:
        for row in sheet.iter_rows(min_row=2, 
                                   max_col=sheet.max_column, 
                                   max_row=sheet.max_row, 
                                   min_col = sheet.max_column-1):
            for cell in row:
                cell.fill = PatternFill(start_color='00FFFFFF', end_color='00FFFFFF',fill_type='solid')
                cell.alignment = Alignment(horizontal='left')
                cell.font = Font(color = 'A000A0')
                
    # the columns fields (country & feedr)
    for row in sheet.iter_rows(min_row=2, 
                               max_col=sheet.max_column, 
                               min_col = 4 if tid not in ['EFx', 'FDR']  else 5, 
                               max_row=3):
        for cell in row:
            cell.fill = colFill
            cell.alignment = Alignment(horizontal='left')
    # the index fields description(odm data fields)
    for row in sheet.iter_rows(min_row=3, 
                               max_col= 3 if tid not in ['EFx', 'FDR'] else 4, 
                               min_col = 1 , 
                               max_row=sheet.max_row):
        if row[2].value == 'PI':
            for cell in row:
                cell.fill = lightRedFill
                cell.alignment = Alignment(horizontal='left')
        elif row[2].value == 'SPI':
            for cell in row:
                cell.fill = redFill
                cell.font = Font(color = 'FFFFFF')
                cell.alignment = Alignment(horizontal='left')
        else: 
            for cell in row:
                cell.fill = rowFill
                cell.alignment = Alignment(horizontal='left')
    # the data field name , check if business definition is avaiale, if yes, put the hyperlink 
    
    for row in sheet.iter_rows(min_row=3, max_col= 1, min_col = 1 , max_row=sheet.max_row):
        for cell in row:
            if cell.value in busi_field_list:
                cell.hyperlink = odmmeta_api_root.format(cell.value)
                cell.font =Font(color = '0000FF')
    
    # adjust the 2nd column width
    #cell_columns = list(sheet.columns)[2]
    #length = max(len(str(cell.value)) if cell.value is not None else 0 for cell in cell_columns)
    sheet.column_dimensions['B'].width = 37
    sheet.column_dimensions['A'].width = 10
    sheet.freeze_panes = sheet['D4'] if tid not in ['EFx', 'FDR'] else sheet['E4']
            
xfile.save(xlsx_file)

In [ ]:
folder_id = '117380324021'  # METATEST folder
from BOX import box_oauth as box  
client = box.get_box_client()
box.save2box_folder(client, folder_id, xlsx_file) 

In [ ]:
for x in col_list_all:
    print(x)

In [ ]:
client = box.get_box_client()


In [ ]:
filename_source = client.file(602746204741).get().name 

In [ ]:
filename_source

In [ ]:
df_col.loc[(df_col.CTID == 'E01') & (df_col.TCOLNAME.str.contains('Absence'))]

In [ ]:
df = pd.read_excel(client.file('731566115825').content())

In [ ]:
df